# LLM Text Preprocessing Foundations (Embeddings)

## Digital Transformation and Enterprise Solutions (TDSE)

Este laboratorio explora los fundamentos del preprocesamiento textual para Large Language Models (LLMs), siguiendo el Chapter 2 del libro *Build a Large Language Model (From Scratch)* de Sebastian Raschka.

El objetivo es comprender cómo el texto crudo se transforma en representaciones numéricas (embeddings) y por qué estas representaciones permiten a los modelos capturar significado, contexto y relaciones semánticas.


In [ ]:
%pip install numpy matplotlib pandas torch scikit-learn tiktoken

In [1]:
# Cargar el texto fuente
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    text = f.read()

print(text[:500])
print("\nTotal de caracteres:", len(text))


I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no great surprise to me to hear that, in the height of his glory, he had dropped his painting, married a rich widow, and established himself in a villa on the Riviera. (Though I rather thought it would have been Rome or Florence.)

"The height of his glory"--that was what the women called it. I can hear Mrs. Gideon Thwing--his last Chicago sitter--deploring his unaccountable abdication. "Of course it'

Total de caracteres: 20479


### ¿Por qué este paso es importante para los LLMs?

Los modelos de lenguaje no entienden texto directamente. Antes de cualquier aprendizaje, el texto existe únicamente como una secuencia de caracteres.  
Este paso permite inspeccionar el estado más crudo de los datos y entender por qué es necesario transformarlos en estructuras más manejables para una red neuronal.


In [2]:
import re

# Tokenización básica usando expresiones regulares
tokens = re.findall(r"\w+|[^\w\s]", text, re.UNICODE)

print("Primeros 50 tokens:")
print(tokens[:50])
print("\nNúmero total de tokens:", len(tokens))


Primeros 50 tokens:
['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '-', '-', 'though', 'a', 'good', 'fellow', 'enough', '-', '-', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in', 'the', 'height', 'of', 'his', 'glory', ',', 'he', 'had', 'dropped', 'his', 'painting', ',', 'married', 'a', 'rich', 'widow', ',', 'and']

Número total de tokens: 4827


### Importancia de la tokenización

La tokenización define la unidad mínima que el modelo procesará.  
Sin tokens:
- No hay vocabulario
- No existen embeddings
- No puede haber aprendizaje

En sistemas agenticos, una tokenización consistente es crucial para que diferentes componentes interpreten el lenguaje de forma compatible.


In [3]:
# Construcción del vocabulario
vocab = sorted(set(tokens))
vocab_size = len(vocab)

token_to_id = {token: idx for idx, token in enumerate(vocab)}
id_to_token = {idx: token for token, idx in token_to_id.items()}

print("Tamaño del vocabulario:", vocab_size)
print("Ejemplo de mapeo token -> id:")
list(token_to_id.items())[:10]


Tamaño del vocabulario: 1148
Ejemplo de mapeo token -> id:


[('!', 0),
 ('"', 1),
 ("'", 2),
 ('(', 3),
 (')', 4),
 (',', 5),
 ('-', 6),
 ('.', 7),
 (':', 8),
 (';', 9)]

### ¿Por qué crear un vocabulario?

Las redes neuronales solo pueden operar con números.  
El vocabulario permite mapear símbolos lingüísticos a identificadores numéricos, sirviendo como puente entre el lenguaje humano y el álgebra lineal utilizada por los modelos.


In [4]:
def encode(token_list):
    return [token_to_id[token] for token in token_list]

def decode(id_list):
    return [id_to_token[idx] for idx in id_list]

encoded_tokens = encode(tokens)

print("Tokens codificados (primeros 20):")
print(encoded_tokens[:20])

print("\nDecodificación de prueba:")
print(decode(encoded_tokens[:20]))


Tokens codificados (primeros 20):
[53, 44, 163, 1021, 57, 38, 835, 129, 271, 504, 6, 6, 1020, 129, 518, 452, 410, 6, 6, 925]

Decodificación de prueba:
['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '-', '-', 'though', 'a', 'good', 'fellow', 'enough', '-', '-', 'so']


### Codificación y decodificación

Este proceso muestra que un LLM es, en esencia, un traductor entre símbolos y números.  
La inteligencia no surge aquí, sino cuando estos números se transforman en vectores entrenables mediante embeddings.


In [7]:
import tiktoken

# Tokenizador GPT-2 basado en Byte Pair Encoding
tokenizer = tiktoken.get_encoding("gpt2")
bpe_tokens = tokenizer.encode(text)

print("Primeros 50 tokens BPE:")
print(bpe_tokens[:50])
print("\nNúmero total de tokens BPE:", len(bpe_tokens))


Primeros 50 tokens BPE:
[40, 367, 2885, 1464, 1807, 3619, 402, 271, 10899, 2138, 257, 7026, 15632, 438, 2016, 257, 922, 5891, 1576, 438, 568, 340, 373, 645, 1049, 5975, 284, 502, 284, 3285, 326, 11, 287, 262, 6001, 286, 465, 13476, 11, 339, 550, 5710, 465, 12036, 11, 6405, 257, 5527, 27075, 11]

Número total de tokens BPE: 5145


### ¿Por qué usar BPE en LLMs modernos?

La tokenización BPE permite:
- Manejar palabras raras o desconocidas
- Reducir el tamaño del vocabulario
- Mejorar la generalización
- Trabajar a nivel sub-palabra

Por esta razón es ampliamente usada en modelos como GPT.


In [8]:
def create_samples(token_ids, max_length, stride):
    samples = []
    for i in range(0, len(token_ids) - max_length + 1, stride):
        samples.append(token_ids[i:i + max_length])
    return samples


### Ventanas de contexto en LLMs

Los LLMs procesan texto en bloques de tamaño fijo llamados ventanas de contexto.  
Estas ventanas determinan cuánta información puede usar el modelo para predecir el siguiente token.


In [9]:
# Experimento sin solapamiento
max_length = 8
stride = 8

samples_no_overlap = create_samples(bpe_tokens, max_length, stride)
print("Número de samples sin solapamiento:", len(samples_no_overlap))


Número de samples sin solapamiento: 643


In [10]:
# Experimento con solapamiento
max_length = 8
stride = 4

samples_overlap = create_samples(bpe_tokens, max_length, stride)
print("Número de samples con solapamiento:", len(samples_overlap))


Número de samples con solapamiento: 1285


### Experimento: efecto de max_length y stride

Al reducir el stride se incrementa el solapamiento entre ventanas, lo que:
- Preserva continuidad semántica
- Aumenta el número de muestras de entrenamiento
- Mejora el aprendizaje contextual

Este trade-off es fundamental en el entrenamiento de LLMs reales.


### Why do embeddings encode meaning, and how are they related to neural networks?

Los embeddings son matrices entrenables que transforman tokens discretos en vectores densos continuos.  
Durante el entrenamiento, el backpropagation ajusta estos vectores para que tokens usados en contextos similares queden cercanos en el espacio vectorial.

Desde la perspectiva de redes neuronales:
- Un embedding es una lookup table diferenciable
- Es equivalente a una capa lineal sin bias
- El significado emerge como geometría en el espacio vectorial

El lenguaje no se programa: se aprende mediante optimización.


## Conclusiones

- El preprocesamiento textual es la base de cualquier LLM
- La tokenización define el lenguaje interno del modelo
- Las ventanas de contexto limitan y estructuran el aprendizaje
- El solapamiento mejora la continuidad semántica
- Los embeddings conectan lenguaje y álgebra lineal

Este laboratorio demuestra cómo decisiones aparentemente simples tienen un impacto profundo en sistemas de IA modernos y soluciones empresariales.


**Curso:** Digital Transformation and Enterprise Solutions (TDSE)  
**Laboratorio:** LLM Text Preprocessing Foundations (Embeddings)
